In [ ]:
import warnings
import datetime as dt
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Metatrader_WINN.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$N'], TFRAMES=['W1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    WEEK_DAY = { 'Sunday':1, 'Monday':2, 'Tuesday':3, 'Wednesday':4, 'Thursday':5, 'Friday':6, 'Saturday':7 }

    Src = pd.DataFrame()
    Src[['A','Z']]                      = Query[['a','z']]
    Src[['Symbol','TF']]                = Query[['symbol','tf']]

    Src['Datetime']                     = Query['datetime']
    Src['Date']                         = Query['datetime'].dt.date
    Src['Time']                         = Query['datetime'].dt.time
    Src['Week Day']                     = Query['datetime'].dt.day_name().map(WEEK_DAY)

    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]

    Src['Change']   = Src['Close'] - Src['Open']
    Src['Variat']   = Src['Close'] / Src['Open'] *100-100
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)

    # ================ Current ================ #
    for x in [0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00]:
        Calc[f'Variat > +{x}%'] = (Calc['Variat'] > +x/100)                 .astype(int)
        Calc[f'Variat < -{x}%'] = (Calc['Variat'] < -x/100)                 .astype(int)


    # ================ Forward ================ #
    Calc[f'Variat +{1}']        =  Calc['Variat'].shift(-1)
    Calc[f'Variat +{1} Pos']    = (Calc[f'Variat +{1}'] > 0)                .astype(int)
    Calc[f'Variat +{1} Neg']    = (Calc[f'Variat +{1}'] < 0)                .astype(int)

    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [10]:
def STATS_UP(Calc):
    pipe = []
    for x in sorted([0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00], reverse=1):
        Df = Calc[Calc[f'Variat > +{x}%']==1]

        pipe.append({
            'Count':            len(Df),
            'Variat > %':       x,
            'Variat +1 Pos':    np.round(Df['Variat +1 Pos'].mean() *100, 1), 
            'Variat +1 Neg':    np.round(Df['Variat +1 Neg'].mean() *100, 1), 
        })
    pass
    return pd.DataFrame(pipe)

STATS_UP(Calc)

,Count,Variat > %,Variat +1 Pos,Variat +1 Neg
0,125,5.0,52.8,46.4
1,125,4.5,52.8,46.4
2,126,4.0,52.4,46.8
3,126,3.5,52.4,46.8
4,126,3.0,52.4,46.8
5,126,2.5,52.4,46.8
6,126,2.0,52.4,46.8
7,126,1.5,52.4,46.8
8,127,1.0,52.0,47.2
9,127,0.5,52.0,47.2


In [11]:
def STATS_DOWN(Calc):
    pipe = []
    for x in sorted([0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00], reverse=0):
        Df = Calc[Calc[f'Variat < -{x}%']==1]

        pipe.append({
            'Count':            len(Df),
            'Variat < %':       -x,
            'Variat +1 Pos':    np.round(Df['Variat +1 Pos'].mean() *100, 1), 
            'Variat +1 Neg':    np.round(Df['Variat +1 Neg'].mean() *100, 1), 
        })
    pass
    return pd.DataFrame(pipe)

STATS_DOWN(Calc)

,Count,Variat < %,Variat +1 Pos,Variat +1 Neg
0,133,-0.5,45.1,54.1
1,133,-1.0,45.1,54.1
2,133,-1.5,45.1,54.1
3,133,-2.0,45.1,54.1
4,133,-2.5,45.1,54.1
5,132,-3.0,45.5,53.8
6,132,-3.5,45.5,53.8
7,132,-4.0,45.5,53.8
8,132,-4.5,45.5,53.8
9,130,-5.0,45.4,53.8


# Snippets

In [12]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src['Change']   = Src['Close'] - Src['Open']
    Src['Variat']   = Src['Close'] / Src['Open'] *100-100
    return Src 


def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    # ================ Current ================ #
    for x in [0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00]:
        Calc[f'Variat > +{x}%'] = (Calc['Variat'] > +x/100)                 .astype(int)
        Calc[f'Variat < -{x}%'] = (Calc['Variat'] < -x/100)                 .astype(int)

    # ================ Forward ================ #
    Calc[f'Variat +{1}']        =  Calc['Variat'].shift(-1)
    Calc[f'Variat +{1} Pos']    = (Calc[f'Variat +{1}'] > 0)                .astype(int)
    Calc[f'Variat +{1} Neg']    = (Calc[f'Variat +{1}'] < 0)                .astype(int)
    return Calc

In [13]:
def STATS_UP(Calc):
    pipe = []
    for x in sorted([0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00], reverse=1):
        Df = Calc[Calc[f'Variat > +{x}%']==1]

        pipe.append({
            'Count':            len(Df),
            'Variat > %':       x,
            'Variat +1 Pos':    np.round(Df['Variat +1 Pos'].mean() *100, 1), 
            'Variat +1 Neg':    np.round(Df['Variat +1 Neg'].mean() *100, 1), 
        })
    pass
    return pd.DataFrame(pipe)


def STATS_DOWN(Calc):
    pipe = []
    for x in sorted([0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00], reverse=0):
        Df = Calc[Calc[f'Variat < -{x}%']==1]

        pipe.append({
            'Count':            len(Df),
            'Variat < %':       -x,
            'Variat +1 Pos':    np.round(Df['Variat +1 Pos'].mean() *100, 1), 
            'Variat +1 Neg':    np.round(Df['Variat +1 Neg'].mean() *100, 1), 
        })
    pass
    return pd.DataFrame(pipe)